seleniumのchromedriverでサイトから情報抽出

In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import re, time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import font_manager as fm
import re
from selenium.common.exceptions import InvalidSessionIdException
import time
from requests.exceptions import ReadTimeout
from selenium.common.exceptions import TimeoutException, WebDriverException




/Users/iinoshumpei/Desktop/test/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
URL = "https://suumo.jp/chintai/soba/tokyo/ensen/"

def build_driver(headless=True):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=opts)

def scrape_suumo_lines_with_selenium(headless=True):
    driver = build_driver(headless=headless)
    wait = WebDriverWait(driver, 20)
    try:
        driver.get(URL)
        # テーブルの見出しが出るまで待機
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "th.searchtable-title")))

        result = {}

        # すべての見出し(th)を順に処理
        headings = driver.find_elements(By.CSS_SELECTOR, "th.searchtable-title")
        for th in headings:
            section = th.text.strip()
            # 同じ行の右隣の td
            td = th.find_element(By.XPATH, "./following-sibling::td[1]")
            # その中のリンク文字列を収集
            names = [a.text.strip() for a in td.find_elements(By.CSS_SELECTOR, "ul.searchitem-list a")]
            result[section] = [n for n in names if n]  # 空文字は除外

        return result

    finally:
        driver.quit()


In [3]:
# 使い方
data = scrape_suumo_lines_with_selenium(headless=True)

# 1) (上位, 下位) のペアを作って MultiIndex を組む
cols = pd.MultiIndex.from_tuples(
    [(k, v) for k, vs in data.items() for v in vs],
    names=["鉄道", "路線"]
)

# 2) その列を持つ空の DataFrame を作成（行は必要に応じて後で追加）
use_data = pd.DataFrame(columns=cols)

The chromedriver version (141.0.7390.76) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (142.0.7444.175); currently, chromedriver 142.0.7444.175 is recommended for chrome 142.*, so it is advised to delete the driver in PATH and retry


In [6]:
use_data.columns.get_level_values(0)
use_data_small = use_data.loc[:, use_data.columns.get_level_values(0).isin(["JR", "東京メトロ", "西武鉄道", "東急電鉄", "都営地下鉄"])]


In [7]:
use_data_small


Empty DataFrame
Columns: [(JR, ＪＲ山手線), (JR, ＪＲ京浜東北線), (JR, ＪＲ東海道本線), (JR, ＪＲ常磐線), (JR, ＪＲ南武線), (JR, ＪＲ横浜線), (JR, ＪＲ横須賀線), (JR, ＪＲ中央線), (JR, ＪＲ青梅線), (JR, ＪＲ五日市線), (JR, ＪＲ武蔵野線), (JR, ＪＲ八高線), (JR, ＪＲ埼京線), (JR, ＪＲ高崎線), (JR, ＪＲ宇都宮線), (JR, ＪＲ総武線), (JR, ＪＲ総武線快速), (JR, ＪＲ京葉線), (JR, 湘南新宿ライン宇須), (JR, 湘南新宿ライン高海), (JR, 東海道新幹線), (JR, 上越新幹線), (JR, 北陸新幹線), (JR, 東北新幹線), (東京メトロ, 東京メトロ銀座線), (東京メトロ, 東京メトロ丸ノ内線), (東京メトロ, 東京メトロ日比谷線), (東京メトロ, 東京メトロ東西線), (東京メトロ, 東京メトロ千代田線), (東京メトロ, 東京メトロ有楽町線), (東京メトロ, 東京メトロ半蔵門線), (東京メトロ, 東京メトロ南北線), (東京メトロ, 東京メトロ副都心線), (西武鉄道, 西武有楽町線), (西武鉄道, 西武新宿線), (西武鉄道, 西武池袋線), (西武鉄道, 西武拝島線), (西武鉄道, 西武山口線), (西武鉄道, 西武国分寺線), (西武鉄道, 西武多摩川線), (西武鉄道, 西武多摩湖線), (西武鉄道, 西武西武園線), (西武鉄道, 西武豊島線), (東急電鉄, 東急東横線), (東急電鉄, 東急田園都市線), (東急電鉄, 東急池上線), (東急電鉄, 東急目黒線), (東急電鉄, 東急多摩川線), (東急電鉄, 東急大井町線), (東急電鉄, 東急世田谷線), (都営地下鉄, 都営浅草線), (都営地下鉄, 都営三田線), (都営地下鉄, 都営新宿線), (都営地下鉄, 都営大江戸線), (都営地下鉄, 都電荒川線), (都営地下鉄, 日暮里・舎人ライナー)]
Index: []

[0 rows x 56 columns]

In [9]:
# JRの路線名から"JR"の文字列のみを削除する
new_columns = []
for rail, route in use_data_small.columns:
    if rail == "JR":
        # 路線名の"JR"の部分のみ削除（全角・半角に対応）
        new_route = route.replace("JR", "").replace("ＪＲ", "").strip()
        new_columns.append((rail, new_route))
    else:
        new_columns.append((rail, route))
use_data_small.columns = pd.MultiIndex.from_tuples(new_columns, names=use_data_small.columns.names)
use_data_small

Empty DataFrame
Columns: [(JR, 山手線), (JR, 京浜東北線), (JR, 東海道本線), (JR, 常磐線), (JR, 南武線), (JR, 横浜線), (JR, 横須賀線), (JR, 中央線), (JR, 青梅線), (JR, 五日市線), (JR, 武蔵野線), (JR, 八高線), (JR, 埼京線), (JR, 高崎線), (JR, 宇都宮線), (JR, 総武線), (JR, 総武線快速), (JR, 京葉線), (JR, 湘南新宿ライン宇須), (JR, 湘南新宿ライン高海), (JR, 東海道新幹線), (JR, 上越新幹線), (JR, 北陸新幹線), (JR, 東北新幹線), (東京メトロ, 東京メトロ銀座線), (東京メトロ, 東京メトロ丸ノ内線), (東京メトロ, 東京メトロ日比谷線), (東京メトロ, 東京メトロ東西線), (東京メトロ, 東京メトロ千代田線), (東京メトロ, 東京メトロ有楽町線), (東京メトロ, 東京メトロ半蔵門線), (東京メトロ, 東京メトロ南北線), (東京メトロ, 東京メトロ副都心線), (西武鉄道, 西武有楽町線), (西武鉄道, 西武新宿線), (西武鉄道, 西武池袋線), (西武鉄道, 西武拝島線), (西武鉄道, 西武山口線), (西武鉄道, 西武国分寺線), (西武鉄道, 西武多摩川線), (西武鉄道, 西武多摩湖線), (西武鉄道, 西武西武園線), (西武鉄道, 西武豊島線), (東急電鉄, 東急東横線), (東急電鉄, 東急田園都市線), (東急電鉄, 東急池上線), (東急電鉄, 東急目黒線), (東急電鉄, 東急多摩川線), (東急電鉄, 東急大井町線), (東急電鉄, 東急世田谷線), (都営地下鉄, 都営浅草線), (都営地下鉄, 都営三田線), (都営地下鉄, 都営新宿線), (都営地下鉄, 都営大江戸線), (都営地下鉄, 都電荒川線), (都営地下鉄, 日暮里・舎人ライナー)]
Index: []

[0 rows x 56 columns]

In [12]:
def add_items_under_each_route(df: pd.DataFrame, items) -> pd.DataFrame:

    """
    既存の列が
      - 2階層: (鉄道, 路線)
      - 3階層: (鉄道, 路線, 項目)
    のいずれでも、各(鉄道, 路線)の下に `items` を用意した3階層カラムへ整形する。

    返り値: 整形済みの新しい DataFrame
    """
    if df.columns.nlevels == 2:
        # 2階層 → 3階層化（各路線に items を展開）
        rails_routes = list(df.columns)  # (鉄道, 路線) のタプル
        new_cols = pd.MultiIndex.from_tuples(
            [(a, b, item) for (a, b) in rails_routes for item in items],
            names=["鉄道", "路線", "項目"]
        )
        # 既存値を引き継ぐ（既存列は項目が空だった想定なのでNaNで埋める）
        out = pd.DataFrame(index=df.index, columns=new_cols, dtype=object)
        return out

    elif df.columns.nlevels == 3:
        # すでに3階層：各(鉄道, 路線)について items を欠けなく揃える
        out = df.copy()
        out.columns = out.columns.set_names(["鉄道", "路線", "項目"])
        cur_tuples = set(out.columns)
        need = []
        # 既存の (鉄道, 路線) セットを作る
        rails_routes = sorted(set((a, b) for (a, b, _) in out.columns))
        for a, b in rails_routes:
            for item in items:
                tup = (a, b, item)
                if tup not in cur_tuples:
                    need.append(tup)
        if need:
            # 欠けている列を追加（NaNで）
            add_df = pd.DataFrame(index=out.index, columns=pd.MultiIndex.from_tuples(need, names=out.columns.names))
            out = pd.concat([out, add_df], axis=1)
            out = out.sort_index(axis=1)
        return out

    else:
        raise ValueError("列の階層は2または3である必要があります。")

# 使い方：あなたの表示している空の 2階層DFに対して
df_new = add_items_under_each_route(use_data_small, items=("駅", "家賃相場(万円)","神谷町までの時間(分)","乗換回数"
))



In [13]:
df_new

Empty DataFrame
Columns: [(JR, 山手線, 駅), (JR, 山手線, 家賃相場(万円)), (JR, 山手線, 神谷町までの時間(分)), (JR, 山手線, 乗換回数), (JR, 京浜東北線, 駅), (JR, 京浜東北線, 家賃相場(万円)), (JR, 京浜東北線, 神谷町までの時間(分)), (JR, 京浜東北線, 乗換回数), (JR, 東海道本線, 駅), (JR, 東海道本線, 家賃相場(万円)), (JR, 東海道本線, 神谷町までの時間(分)), (JR, 東海道本線, 乗換回数), (JR, 常磐線, 駅), (JR, 常磐線, 家賃相場(万円)), (JR, 常磐線, 神谷町までの時間(分)), (JR, 常磐線, 乗換回数), (JR, 南武線, 駅), (JR, 南武線, 家賃相場(万円)), (JR, 南武線, 神谷町までの時間(分)), (JR, 南武線, 乗換回数), (JR, 横浜線, 駅), (JR, 横浜線, 家賃相場(万円)), (JR, 横浜線, 神谷町までの時間(分)), (JR, 横浜線, 乗換回数), (JR, 横須賀線, 駅), (JR, 横須賀線, 家賃相場(万円)), (JR, 横須賀線, 神谷町までの時間(分)), (JR, 横須賀線, 乗換回数), (JR, 中央線, 駅), (JR, 中央線, 家賃相場(万円)), (JR, 中央線, 神谷町までの時間(分)), (JR, 中央線, 乗換回数), (JR, 青梅線, 駅), (JR, 青梅線, 家賃相場(万円)), (JR, 青梅線, 神谷町までの時間(分)), (JR, 青梅線, 乗換回数), (JR, 五日市線, 駅), (JR, 五日市線, 家賃相場(万円)), (JR, 五日市線, 神谷町までの時間(分)), (JR, 五日市線, 乗換回数), (JR, 武蔵野線, 駅), (JR, 武蔵野線, 家賃相場(万円)), (JR, 武蔵野線, 神谷町までの時間(分)), (JR, 武蔵野線, 乗換回数), (JR, 八高線, 駅), (JR, 八高線, 家賃相場(万円)), (JR, 八高線, 神谷町までの時間(分)), (JR, 八高線, 乗換回数), (JR, 埼京線, 駅), (JR, 埼京線, 家賃相場(万円)), (JR, 埼京線, 神谷町までの時間(分)), (JR, 埼京線, 乗換回数), (JR, 高崎線, 駅), (JR, 高崎線, 家賃相場(万円)), (JR, 高崎線, 神谷町までの時間(分)), (JR, 高崎線, 乗換回数), (JR, 宇都宮線, 駅), (JR, 宇都宮線, 家賃相場(万円)), (JR, 宇都宮線, 神谷町までの時間(分)), (JR, 宇都宮線, 乗換回数), (JR, 総武線, 駅), (JR, 総武線, 家賃相場(万円)), (JR, 総武線, 神谷町までの時間(分)), (JR, 総武線, 乗換回数), (JR, 総武線快速, 駅), (JR, 総武線快速, 家賃相場(万円)), (JR, 総武線快速, 神谷町までの時間(分)), (JR, 総武線快速, 乗換回数), (JR, 京葉線, 駅), (JR, 京葉線, 家賃相場(万円)), (JR, 京葉線, 神谷町までの時間(分)), (JR, 京葉線, 乗換回数), (JR, 湘南新宿ライン宇須, 駅), (JR, 湘南新宿ライン宇須, 家賃相場(万円)), (JR, 湘南新宿ライン宇須, 神谷町までの時間(分)), (JR, 湘南新宿ライン宇須, 乗換回数), (JR, 湘南新宿ライン高海, 駅), (JR, 湘南新宿ライン高海, 家賃相場(万円)), (JR, 湘南新宿ライン高海, 神谷町までの時間(分)), (JR, 湘南新宿ライン高海, 乗換回数), (JR, 東海道新幹線, 駅), (JR, 東海道新幹線, 家賃相場(万円)), (JR, 東海道新幹線, 神谷町までの時間(分)), (JR, 東海道新幹線, 乗換回数), (JR, 上越新幹線, 駅), (JR, 上越新幹線, 家賃相場(万円)), (JR, 上越新幹線, 神谷町までの時間(分)), (JR, 上越新幹線, 乗換回数), (JR, 北陸新幹線, 駅), (JR, 北陸新幹線, 家賃相場(万円)), (JR, 北陸新幹線, 神谷町までの時間(分)), (JR, 北陸新幹線, 乗換回数), (JR, 東北新幹線, 駅), (JR, 東北新幹線, 家賃相場(万円)), (JR, 東北新幹線, 神谷町までの時間(分)), (JR, 東北新幹線, 乗換回数), (東京メトロ, 東京メトロ銀座線, 駅), (東京メトロ, 東京メトロ銀座線, 家賃相場(万円)), (東京メトロ, 東京メトロ銀座線, 神谷町までの時間(分)), (東京メトロ, 東京メトロ銀座線, 乗換回数), ...]
Index: []

[0 rows x 224 columns]

沿線の選択（例：JR山手線）

In [14]:
START_URL = "https://suumo.jp/chintai/soba/tokyo/ensen/"

In [15]:
URL = ("https://transit.yahoo.co.jp/?from=&to=&fromgid=&togid=&flatlon=&tlatlon=&via=&viacode="
       "&y=2025&m=10&d=19&hh=22&m1=1&m2=1&type=1&ticket=ic&expkind=1&userpass=1&ws=3&s=0"
       "&al=1&shin=1&ex=1&hb=1&lb=1&sr=1")

def build_driver(headless=True):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=opts)

def close_consent_if_any(driver, timeout=4):
    """同意/クッキー系のモーダルがあれば閉じる（無ければスルー）"""
    wait = WebDriverWait(driver, timeout)
    try:
        btn = wait.until(
            EC.element_to_be_clickable((
                By.XPATH,
                "//button[contains(.,'同意')] | //button[contains(.,'OK')] | //a[contains(.,'同意')]"
            ))
        )
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(0.2)
    except Exception:
        pass

def _select_first_station(driver, field: str, keyword: str):
    wait = WebDriverWait(driver, 10)
    label = "出発" if field == "from" else "到着"
    suggest_cls = "from_suggest" if field == "from" else "to_suggest"

    # 入力欄
    input_el = wait.until(EC.presence_of_element_located((
        By.XPATH,
        f"//form[@name='search']//dt[normalize-space()='{label}']"
        f"/following-sibling::dd[1]//input[@name='{field}']"
    )))

    # プレースホルダを念のため隠す
    try:
        dd = input_el.find_element(By.XPATH, "./ancestor::dd[1]")
        ph = dd.find_element(By.CSS_SELECTOR, "span.placeholder")
        driver.execute_script("arguments[0].style.display='none';", ph)
    except Exception:
        pass

    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", input_el)
    input_el.clear()
    input_el.send_keys(keyword)
    time.sleep(0.35)

    # サジェスト表示待機
    wait.until(EC.visibility_of_element_located((
        By.XPATH, f"//dl[@id='suggest' and contains(@class,'{suggest_cls}') and contains(@style,'block')]"
    )))

    # 「鉄道駅」の一番上 → 無ければサジェストの一番上
    xpath_station_top = (
        f"//dl[@id='suggest' and contains(@class,'{suggest_cls}')]"
        "//dt[contains(normalize-space(),'鉄道駅')]/following-sibling::dd[1]//ul/li[1]/a"
    )
    xpath_any_top = (
        f"//dl[@id='suggest' and contains(@class,'{suggest_cls}')]//ul/li[1]/a"
    )
    anchors = driver.find_elements(By.XPATH, xpath_station_top)
    if not anchors:
        anchors = driver.find_elements(By.XPATH, xpath_any_top)
    if not anchors:
        raise RuntimeError("サジェスト候補が見つかりませんでした")
    try:
        anchors[0].click()
    except Exception:
        driver.execute_script("arguments[0].click();", anchors[0])

    # 値が入るまで待つ
    wait.until(lambda d: d.find_element(By.NAME, field).get_attribute("value").strip() != "")
    return driver.find_element(By.NAME, field).get_attribute("value").strip()

def _wait_results_any_key(driver, timeout=10):
    """結果ページの代表要素のいずれかが現れるのを待つ"""
    wait = WebDriverWait(driver, timeout)

    def any_key_present(d):
        css_list = [
            "li.time span.small",                    # 旧/一般的
            "li.time span.mark",                     # バリアント
            "dl a[href^='#route']",                  # ルート見出しアンカー
            "div#rsltlst dl",                        # コンテナ+行
            "section ul.routeDetail, div.routeDetail"
        ]
        for css in css_list:
            if d.find_elements(By.CSS_SELECTOR, css):
                return True
        return False

    wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
    wait.until(any_key_present)

def _extract_duration_and_transfers(driver, timeout):
    """最初のルート行から 所要時間 と 乗換回数 を堅牢に抜く"""
    wait = WebDriverWait(driver, timeout)
    # ページの完全ロード
    wait.until(lambda d: d.execute_script("return document.readyState") == "complete")

    # “時間”キー（どちらか）出現を待つ
    def find_time_span():
        els = driver.find_elements(By.CSS_SELECTOR, "li.time span.small")  # 例: 9分
        if not els:
            els = driver.find_elements(By.CSS_SELECTOR, "li.time span.mark")  # バリアント
        return els[0] if els else None

    end = time.time() + timeout
    el_time = None
    while time.time() < end and el_time is None:
        el_time = find_time_span()
        if not el_time:
            time.sleep(0.3)

    if el_time is None:
        # デバッグ用に保存
        with open("debug_results.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        raise TimeoutError("所要時間の要素が見つかりません（debug_results.html を確認）")

    # ルート行（dl か route の親ブロック）へ遡る
    dl = el_time.find_element(By.XPATH, "./ancestor::dl[1]")

    # 所要時間
    duration_text = el_time.text.strip()                  # 例: "9分" or "13分"
    if not duration_text:
        # バリアントのときは li.time 全体から拾う
        duration_text = dl.find_element(By.CSS_SELECTOR, "li.time").text.strip()

    # 乗換回数：候補を順に試す → 無ければテキストから抽出
    transfers_text = ""
    candidates = [
        "li.transfer span.mark",
        "li.transfer .mark",
        "li.transfer"                  # 例: "乗換：0回"
    ]
    for css in candidates:
        try:
            transfers_text = dl.find_element(By.CSS_SELECTOR, css).text.strip()
            if transfers_text:
                break
        except Exception:
            pass

    # テキストが「乗換：0回」形式なら数字だけ拾う
    if not re.search(r"\d", transfers_text):
        try:
            txt = dl.find_element(By.CSS_SELECTOR, "li.transfer").text
            m = re.search(r"(\d+)\s*回", txt)
            if m:
                transfers_text = m.group(1)
        except Exception:
            pass

    # それでも取れない場合は 0（直通）とみなす
    if not transfers_text:
        transfers_text = "0"

    return duration_text, transfers_text

def search_get_url_and_metrics_core(driver, from_kw: str, to_kw: str,
                                    t_wait=10, t_results=40, t_extract=15):
    wait = WebDriverWait(driver, t_wait)
    driver.get(URL)
    close_consent_if_any(driver)

    _select_first_station(driver, "from", from_kw)
    _select_first_station(driver, "to",   to_kw)

    # 検索
    try:
        btn = wait.until(EC.element_to_be_clickable((By.ID, "searchModuleSubmit")))
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
        try:
            btn.click()
        except Exception:
            driver.execute_script("arguments[0].click();", btn)
    except Exception:
        form = driver.find_element(By.NAME, "search")
        driver.execute_script("arguments[0].submit();", form)

    # 遷移 or 結果DOM
    start = driver.current_url
    WebDriverWait(driver, t_wait).until(lambda d: d.current_url != start or True)
    _wait_results_any_key(driver, timeout=t_results)

    duration, transfers = _extract_duration_and_transfers(driver, timeout=t_extract)
    return driver.current_url, duration, transfers



In [16]:
def search_get_url_and_metrics(from_kw: str, to_kw: str, headless=True, 
                                t_wait=10, t_results=40, t_extract=15):
    """
    search_get_url_and_metrics_core のラッパー関数。
    headless パラメータを受け取り、ドライバーを作成・管理する。
    """
    driver = build_driver(headless=headless)
    try:
        url, duration, transfers = search_get_url_and_metrics_core(
            driver, from_kw, to_kw, t_wait=t_wait, t_results=t_results, t_extract=t_extract
        )
        return url, duration, transfers
    finally:
        driver.quit()

#Driverの設定関数
def build_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    # （Colab なら↓も効くことが多い）
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=options)

#線路選択関数
def get_url_by_click(driver, train_company, line_name) -> str:
    driver.get(START_URL)
    wait = WebDriverWait(driver, 15)

    jr_th = wait.until(EC.presence_of_element_located((
        By.XPATH, f"//th[contains(@class,'searchtable-title')][normalize-space()='{train_company}']"
    )))
    line_link = wait.until(EC.element_to_be_clickable((
        By.XPATH, f".//following-sibling::td[1]//a[contains(normalize-space(.),'{line_name}')]"
    )))
    # 3) クリックして遷移先 URL を取得
    line_link.click()
    # wait.until(EC.url_contains("en_yamanotesen"))
    return driver.current_url
    # --- 更新前の結果テーブルHTMLをスナップショット（AJAX更新待ちに使う） ---
def first_result_table_html(drv):
    try:
        tbl = drv.find_element(By.XPATH, "//table[.//th[contains(.,'駅') or contains(.,'駅名')]]")
        return tbl.get_attribute("outerHTML")
    except Exception:
        return drv.page_source

# リトライを行うためのヘルパ関数
def retry(func, max_retries=3, delay=3, exception_types=(TimeoutException, WebDriverException)):
    for attempt in range(max_retries):
        try:
            return func()
        except exception_types as e:
            print(f"リトライ: {func.__name__} ({attempt+1}/{max_retries}) 理由: {e}")
            time.sleep(delay)
    raise

In [ ]:
# ---- 実行例 ----



for col in use_data_small.columns:
# for col in seibu_columns:
    # use_data.columnsの要素は (train_company, line_name) のタプル想定
    train_company, line_name = col

    driver = build_driver(headless=True)
    wait = WebDriverWait(driver, 15)

    # クリックして取得（リトライあり）
    def _get_url():
        return get_url_by_click(driver, train_company, line_name)
    url_clicked = retry(_get_url, max_retries=3, delay=3)
    print("検索ヒットURL:", url_clicked)

    driver.quit()  # 一旦閉じる

    driver = build_driver(headless=True)
    wait = WebDriverWait(driver, 15)

    # driver.get(url_clicked) も接続エラーのことがあるのでリトライで囲む
    def _driver_get():
        driver.get(url_clicked)
        return True
    retry(_driver_get, max_retries=3, delay=3)

    # --- 建物種別=マンション（id="souba_ts1"）を選択 ---
    mansion_label = retry(lambda: wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "label[for='souba_ts1']"))), max_retries=3, delay=2)
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", mansion_label)
    retry(lambda: wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='souba_ts1']"))).click(), max_retries=3, delay=2)

    # 念のため選択状態を確認（未選択ならJSクリックで再試行）
    mansion_input = retry(lambda: wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "input#souba_ts1[name='ts'][type='radio']"))), max_retries=3, delay=2)
    if not mansion_input.is_selected():
        driver.execute_script("arguments[0].click();", mansion_label)

    # --- 間取り=ワンルーム（id="souba_madori-oneroom"）を選択 ---
    one_room_label = retry(lambda: wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "label[for='souba_madori-oneroom']"))), max_retries=3, delay=2)
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", one_room_label)
    retry(lambda: wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='souba_madori-oneroom']"))).click(), max_retries=3, delay=2)

    # 状態確認（既定でチェック済みでもOK。未選択ならJSクリック）
    one_room_input = retry(lambda: wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "input#souba_madori-oneroom[name='mdKbn'][type='radio']"))), max_retries=3, delay=2)
    if not one_room_input.is_selected():
        driver.execute_script("arguments[0].click();", one_room_label)

    before_html = first_result_table_html(driver)

    # --- 「相場情報を更新する」アンカーをクリック（AJAX更新想定） ---
    koshin_link = retry(lambda: wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, "a.js-sobaKoshinLink"))), max_retries=3, delay=2)
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", koshin_link)
    try:
        koshin_link.click()
    except Exception:
        driver.execute_script("arguments[0].click();", koshin_link)

    # URLは変わらないことが多いので、DOMの変化で待つ
    retry(lambda: wait.until(lambda d: first_result_table_html(d) != before_html), max_retries=3, delay=2)

    # 駅×家賃のテーブル行（AJAX反映済みを想定）
    def _wait_rows():
        return wait.until(EC.presence_of_all_elements_located((
            By.XPATH,
            "//table[.//th[contains(.,'駅')]][.//th[contains(.,'家賃')]]"
            "//tr[contains(@class,'js-graph-data')]"
        )))
    rows = retry(_wait_rows, max_retries=3, delay=2)

    pairs = []
    for tr in rows:
        # 駅名（1列目の <a>。無ければセルテキスト）
        try:
            station = tr.find_element(By.XPATH, ".//td[1]//a").text.strip()
        except:
            station = tr.find_element(By.XPATH, ".//td[1]").text.strip()

        # 数値部（強調スパン or data-value）
        try:
            num = tr.find_element(
                By.XPATH,
                ".//td[contains(@class,'graphpanel_matrix-td_graphinfo')]"
                "//span[contains(@class,'graphpanel_matrix-td_graphinfo-strong')]"
            ).text.strip()
        except:
            num = (tr.get_attribute("data-value") or "").strip()

        rent_text = f"{num}万円" if num else tr.find_element(
            By.XPATH,
            ".//td[contains(@class,'graphpanel_matrix-td_graphinfo')]"
        ).text.strip()

        pairs.append((station, rent_text))

    df = pd.DataFrame(pairs, columns=["駅", "家賃相場(万円)"])
    df["家賃相場(万円)"] = df["家賃相場(万円)"].str.replace("万円", "", regex=False)
    df["家賃相場(万円)"] = pd.to_numeric(df["家賃相場(万円)"], errors="coerce")  # dtype: float64

    df["神谷町までの時間(分)"] = None
    df["乗換回数"] = None

    for index in range(len(df)):
        df_new.loc[index, (train_company, line_name, '駅')] = df['駅'][index]
        df_new.loc[index, (train_company, line_name, '家賃相場(万円)')] = df['家賃相場(万円)'][index]

        from_line = df["駅"][index]
        to_line = "神谷町"

        if df["駅"][index] == to_line:
            df["神谷町までの時間(分)"][index] = 0
            df["乗換回数"][index] = 0
            
            df_new.loc[index, (train_company, line_name, '神谷町までの時間(分)')] = df['神谷町までの時間(分)'][index]
            df_new.loc[index, (train_company, line_name, '乗換回数')] = df['乗換回数'][index]
        else:
            # ----- search_get_url_and_metrics もリトライで囲みます -----
            def _search():
                return search_get_url_and_metrics(from_line, to_line, headless=True)
            url, duration, transfers = retry(_search, max_retries=3, delay=4)
            # --------------------------------------------------------

            if isinstance(transfers, str) and "乗換：" in transfers:
                transfers = transfers.replace("乗換：", "")
            transfers = transfers.replace("回", "")
            # durationの文字列を時間の分に変え、数値とするコード

            if isinstance(duration, str):
                # 例: "1時間12分"や"55分"などに対応
                hour_match = re.search(r'(\d+)\s*時間', duration)
                min_match = re.search(r'(\d+)\s*分', duration)
                total_minutes = 0
                if hour_match:
                    total_minutes += int(hour_match.group(1)) * 60
                if min_match:
                    total_minutes += int(min_match.group(1))
                duration = total_minutes if total_minutes > 0 else None

            df["神谷町までの時間(分)"][index] = duration
            df["乗換回数"][index] = transfers

            df_new.loc[index, (train_company, line_name, '神谷町までの時間(分)')] = df['神谷町までの時間(分)'][index]
            df_new.loc[index, (train_company, line_name, '乗換回数')] = df['乗換回数'][index]

    driver.quit()



In [19]:
df_new

鉄道        JR                                                                \
路線       山手線                               京浜東北線                             
項目         駅 家賃相場(万円) 神谷町までの時間(分) 乗換回数         駅 家賃相場(万円) 神谷町までの時間(分) 乗換回数   
0         東京      9.8          14    1        赤羽      7.0          41    1   
1        有楽町     12.3          12    0       東十条      6.6          42    2   
2         新橋     11.9          14    1        王子      6.9          41    2   
3        浜松町     11.2          23    1       上中里      7.0          44    2   
4         田町     10.3          27    1        田端      7.3          34    1   
5   高輪ゲートウェイ      9.5          29    1      西日暮里      7.5          28    1   
6         品川      9.4          27    1       日暮里      7.9          31    1   
7         大崎      8.5          20    1        鶯谷      7.9          31    1   
8        五反田      9.0          21    1        上野      8.5          24    0   
9         目黒     10.0          19    1       御徒町      8.9          28    0   
10       恵比寿     11.4           9    0       秋葉原      9.6          20    0   
11        渋谷     11.4          19    1        神田     10.6          20    1   
12        原宿     11.9          24    1        東京      9.8          14    1   
13       代々木     10.0          21    1       有楽町     12.3          12    0   
14        新宿      8.4          23    1        新橋     11.9          14    1   
15      新大久保      8.0          28    1       浜松町     11.2          23    1   
16      高田馬場      7.7          31    1        田町     10.3          27    1   
17        目白      7.4          33    1  高輪ゲートウェイ      9.5          29    1   
18        池袋      7.5          34    1        品川      9.4          27    1   
19        大塚      7.6          38    1       大井町      8.1          27    1   
20        巣鴨      7.8          28    1        大森      7.3          35    2   
21        駒込      7.5          36    1        蒲田      7.5          39    2   
22        田端      7.3          34    1        川崎      6.4          38    2   
23      西日暮里      7.5          28    1        鶴見      5.9          47    1   
24       日暮里      7.9          31    1       新子安      5.0          51    1   
25        鶯谷      7.9          31    1      東神奈川      6.3          54    1   
26        上野      8.5          24    0        横浜      6.8          38    1   
27       御徒町      8.9          28    0       桜木町      7.0          54    2   
28       秋葉原      9.6          20    0        関内      6.9          59    3   
29        神田     10.6          20    1       石川町      6.2          61    3   
30       NaN      NaN         NaN  NaN        山手      5.2          61    2   
31       NaN      NaN         NaN  NaN        根岸      4.3          63    2   
32       NaN      NaN         NaN  NaN        磯子      4.0          73    2   
33       NaN      NaN         NaN  NaN       新杉田      4.5          76    2   
34       NaN      NaN         NaN  NaN       洋光台      4.0          79    1   
35       NaN      NaN         NaN  NaN       港南台      4.6          82    1   
36       NaN      NaN         NaN  NaN       本郷台      5.0          82    3   
37       NaN      NaN         NaN  NaN        大船      5.5          62    1   
38       NaN      NaN         NaN  NaN       NaN      NaN         NaN  NaN   
39       NaN      NaN         NaN  NaN       NaN      NaN         NaN  NaN   
40       NaN      NaN         NaN  NaN       NaN      NaN         NaN  NaN   
41       NaN      NaN         NaN  NaN       NaN      NaN         NaN  NaN   
42       NaN      NaN         NaN  NaN       NaN      NaN         NaN  NaN   
43       NaN      NaN         NaN  NaN       NaN      NaN         NaN  NaN   
44       NaN      NaN         NaN  NaN       NaN      NaN         NaN  NaN   
45       NaN      NaN         NaN  NaN       NaN      NaN         NaN  NaN   
46       NaN      NaN         NaN  NaN       NaN      NaN         NaN  NaN   
47       NaN      NaN         NaN  NaN       NaN      NaN         NaN  NaN   
48       NaN      NaN 

選択した沿線の各駅ごとの家賃相場（建物種別：マンション、間取り：ワンルーム選択）

乗換乗車サイトから時間取得

In [20]:
df_new.to_csv("/Users/iinoshumpei/Desktop/output.csv", index=True, encoding="utf-8-sig")

In [21]:
fix_data = pd.read_csv("/Users/iinoshumpei/Desktop/output.csv", header=[0, 1, 2])
# 列ラベルは ('JR', '山手線', '駅') みたいなタプルになっている


In [22]:
# ② JSONファイルとして書き出す
fix_data.to_json("/Users/iinoshumpei/Desktop/test/fix_data.json", orient="records", force_ascii=False)

In [31]:
import json

with open("/Users/iinoshumpei/Desktop/test/fix_data.json", "r", encoding="utf-8") as f:
    fix_data_loaded = json.load(f)



In [44]:
# (任意, 任意, '神谷町までの時間(分)')として、その値が130以上のものを('鉄道', '沿線', '神谷町までの時間'): 値 で出力
for row in fix_data_loaded:
    for key_str, value in row.items():
        # キーが"('鉄道', '沿線', '神谷町までの時間(分)')"の形式か
        if '神谷町までの時間(分)' in key_str and value is not None:
            try:
                v = float(value)
            except (ValueError, TypeError):
                continue
            if v >= 130:
                print(f"{key_str}：{v}")

('JR', '湘南新宿ライン高海', '神谷町までの時間(分)')：131.0
('JR', '湘南新宿ライン高海', '神谷町までの時間(分)')：379.0
('JR', '八高線', '神谷町までの時間(分)')：400.0
('JR', '湘南新宿ライン高海', '神谷町までの時間(分)')：374.0
('東京メトロ', '東京メトロ銀座線', '神谷町までの時間(分)')：537.0
('JR', '八高線', '神谷町までの時間(分)')：447.0
('JR', '八高線', '神谷町までの時間(分)')：402.0
('JR', '八高線', '神谷町までの時間(分)')：408.0
('JR', '青梅線', '神谷町までの時間(分)')：375.0
('JR', '八高線', '神谷町までの時間(分)')：397.0
('JR', '青梅線', '神谷町までの時間(分)')：377.0
('JR', '八高線', '神谷町までの時間(分)')：418.0
('JR', '青梅線', '神谷町までの時間(分)')：379.0
('JR', '八高線', '神谷町までの時間(分)')：132.0
('JR', '青梅線', '神谷町までの時間(分)')：382.0
('JR', '青梅線', '神谷町までの時間(分)')：385.0
('JR', '高崎線', '神谷町までの時間(分)')：374.0
('JR', '青梅線', '神谷町までの時間(分)')：389.0
('JR', '高崎線', '神谷町までの時間(分)')：379.0
('JR', '青梅線', '神谷町までの時間(分)')：393.0
('JR', '宇都宮線', '神谷町までの時間(分)')：469.0
('JR', '青梅線', '神谷町までの時間(分)')：399.0
('JR', '青梅線', '神谷町までの時間(分)')：405.0
('JR', '宇都宮線', '神谷町までの時間(分)')：133.0
('西武鉄道', '西武池袋線', '神谷町までの時間(分)')：381.0
('西武鉄道', '西武池袋線', '神谷町までの時間(分)')：384.0
('西武鉄道', '西武池袋線', '神谷町までの時間(分)')：389.0
('JR', '常磐線', '